In [29]:
import pandas as pd

# 대화 데이터 구성
dialog_data = [
    {"Role": "User", "Contents": "다음학기에 파이썬을 배우려고 하는데, 어떤 책을 추천해주시겠어요?"},
    {"Role": "Model", "Contents": "(Do it!) 점프 투 파이썬 책을 추천합니다."},
    {"Role": "User", "Contents": "이 책의 KDC 랑 대출건수도 알려줄래요?"},
    {"Role": "Model", "Contents": "KDC는 005.133이고 대출건수는 147 입니다."},
    {"Role": "User", "Contents": "C 언어 에 대해 공부하고 싶은데, 책 좀 추천해줘"},
    {"Role": "Model", "Contents": "윤성우의 열혈 C 프로그래밍 책을 추천합니다."},
    {"Role": "User", "Contents": "이 책의 KDC 랑 대출건수도 알려줄래?"},
    {"Role": "Model", "Contents": "KDC는 005.135이고 대출건수는 113 입니다."},
    {"Role": "User", "Contents": "프로젝트 준비로 알고리즘에 대해서 알고싶은데, 어떤 책이 좋을까?"},
    {"Role": "Model", "Contents": "알고리즘 행성 : 쇼핑부터 인공지능까지, 우리 삶을 움직이는 알고리즘에 관한 모든것 책을 추천합니다."},
    {"Role": "User", "Contents": "이 책의 KDC 랑 대출건수도 알려줘"},
    {"Role": "Model", "Contents": "KDC는 005.1이고 대출건수는 133 입니다."},
    {"Role": "User", "Contents": "파이썬 너무 어려운데, 어떤 책이 좋을까"},
    {"Role": "Model", "Contents": "(Do it!) 점프 투 파이썬 책을 추천합니다."},
    {"Role": "User", "Contents": "이 책의 KDC 랑 대출건수도 알려줄래요?"},
    {"Role": "Model", "Contents": "KDC는 005.133이고 대출건수는 147 입니다."},
    {"Role": "User", "Contents": "운영체제 관련책을 추천해줘"},
    {"Role": "Model", "Contents": "(지금 당장 써먹는) 윈도우 10 마스터북 : 진화하는 MS운영체제의 끝판왕 책을 추천합니다."},
    {"Role": "User", "Contents": "이 책의 KDC 랑 대출건수도 알려줄래요?"},
    {"Role": "Model", "Contents": "KDC는 005.44이고 대출건수는 23 입니다."},
]

# 대화 데이터를 데이터프레임으로 변환
df = pd.DataFrame(dialog_data)

# 대화 데이터를 CSV 파일로 저장
df.to_csv("conversation_dataset.csv", index=False)

# 키워드 데이터를 로드

# 열 이름을 리스트로 지정
column_names = ["Keywords"]

# 데이터를 로드할 때 열 이름을 설정
df1 = pd.read_excel('/content/drive/MyDrive/대출 데이터 모음/키워드 처리 폴더/keyword.xlsx', names=column_names)

# 열 이름이 설정된 데이터프레임을 CSV 파일로 저장
df1.to_csv('/keyword_dataset.csv', index=False)


In [30]:
import pandas as pd

# 대화 데이터 로드
conversation_df = pd.read_csv("conversation_dataset.csv")
keywords_df = pd.read_csv("keywords_dataset.csv")

In [31]:
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained("gpt2")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [32]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer

model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="./model_output",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=100,
)


In [33]:
from transformers import AutoTokenizer

# tokenizer 초기화
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# 패딩 토큰 추가
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# preprocess_data 함수 정의
def preprocess_data(data_df):
    input_texts = list(data_df["Contents"])
    input_ids = tokenizer(input_texts, padding=True, truncation=True, return_tensors="pt").input_ids
    return {"input_ids": input_ids}


In [34]:
import pandas as pd

# 열 이름을 "Keywords"로 설정하여 데이터프레임 생성
keywords_df = pd.read_excel('/content/drive/MyDrive/대출 데이터 모음/키워드 처리 폴더/keyword.xlsx', names=["Keywords"])

# 열 이름 확인
print(keywords_df.columns)

# 열 이름이 설정된 데이터프레임을 CSV 파일로 저장
keywords_df.to_csv('keyword_dataset.csv', index=False)


Index(['Keywords'], dtype='object')


In [36]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer와 모델 초기화
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

# 데이터를 토큰화하고 모델 입력 형식에 맞게 변환
def preprocess_data(data_df):
    input_texts = list(data_df["Contents"])
    input_ids = tokenizer(input_texts, padding=True, truncation=True, return_tensors="pt").input_ids
    return {"input_ids": input_ids}

# 대화 데이터를 preprocess
train_dataset = preprocess_data(conversation_df)

# 추천 키워드 데이터를 로드
keywords_df = pd.read_csv("keyword_dataset.csv")

# 대화 데이터와 키워드 데이터의 적절한 매핑을 만듭니다.
# 여기서는 대화 데이터의 인덱스를 키워드 데이터에 매핑하는 예제를 제공합니다.
def preprocess_keywords(data_df):
    label_texts = list(data_df["Keywords"])
    return {"labels": label_texts}

# 키워드 데이터를 preprocess
test_dataset = preprocess_keywords(keywords_df)



ValueError: ignored